In [3]:
%run ../tools/autoipy.py

from analyze_orbits import get_orbits
# from matplotlib.ticker import FormatStrFormatter
import collections
import numpy as np
import math
import emcee
import pickle
import os
import traceback
from profilestats import profile
from pathlib import Path
from scipy.stats import rayleigh
from scipy.stats import beta
from sklearn.externals import joblib

# FIXED VALUES

mass_sun=1 # solar mass
mass_pl=1 # earth mass

mass_sun_kg = 1.989e30
mass_earth_kg = 5.972e24
G_const = 6.67408e-11
mu_earth = G_const * (mass_sun_kg + mass_earth_kg)
AU2m = 1.496e11

f_disp_e = 0.081 # dispersion parameter for ecc distribution
C_norm_e = ( 1 - np.exp(-1/(2*f_disp_e**2)) )**-1


lna_min = np.log(0.1) 
lna_max = np.log(50)

# lna_min = np.log(0.2) 
# lna_max = np.log(1.71)

beta_a = 0.867 # kipping 2013 http://adsabs.harvard.edu/cgi-bin/bib_query?arXiv:1306.4982
beta_b = 3.03


lna_min_true = np.log(0.95) #lna_max_true = np.log(1.70) 
lna_max_true = np.log(1.70) #lna_min_true = np.log(0.95)

importing Jupyter notebook from analyze_orbits.ipynb


In [4]:
# emcee functions for Rayleigh distribution

from numpy import log, array, pi, inf, exp, sqrt

def lnprior_Ray(theta):
    # The parameters are stored as a vector of values, so unpack them
    lna, cosi, ecc, omega_p, xi_0, lan = theta
    
    # let eccentricity be flat for now
    if not (lna_min < lna < lna_max) or not (-1 < cosi < 1) or not (0 < omega_p < 2*pi) or not (0 < xi_0 < 2*pi) or not (0 < lan < 2*pi):
        return -inf
    if not (0 <= ecc < 1): # bounds of rayleigh distribution
        return -inf
     
    
    lnPr_ecc = log(C_norm_e*ecc/f_disp_e**2) - ecc**2/(2*f_disp_e**2) # rayleigh distribution for ecc prior

    return lnPr_ecc  # this is normalized (area under pdf = 1)


def lnlike(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet, a_IWA):
    # fitting aproj(xi) with several measurements of aproj given as a list
    
    x_model, y_model, aproj_model = obv_at_epoch(theta, range(n_epochs), cadence, noise=False)  
    
    
    try:
        false_nondet_model = [(aproj_model[i] > a_IWA) for i in i_nondet] 
        reject = any(false_nondet_model) 
    except IndexError: # first epoch
        false_nondet_model = ((aproj_model > a_IWA) and (0 in i_nondet))
        reject = false_nondet_model 
     
    if reject: # any model predictions are outside IWA for nondetection epochs
#         print('rejecting aproj_model', aproj_model, 'with i_nondet', i_nondet)
        return -inf

    ll_nondet = 0 
#     n_nondet_model = len(i_nondet) # number of correct nondetections in model is necessarily all of them
#     ll_nondet = -n_nondet_model * log(a_IWA - exp(lna_min))
    

    # only fit params with aproj > IWA
    X = [x[i] for i in i_det]
    Y = [y[i] for i in i_det]
    
    try:
        X_model = [x_model[i] for i in i_det]
        Y_model = [y_model[i] for i in i_det]
    except IndexError: # if first epoch
        X_model = [[x_model][i] for i in i_det]
        Y_model = [[y_model][i] for i in i_det]
        
    chi2 =  np.sum(  ( ( array(X) - array(X_model) )**2 + ( array(Y) - array(Y_model) )**2 )/(s**2) )
    ll_det = -chi2 - len(i_det)*( log(pi) + 2*log(s) )
    
    return  ll_det + ll_nondet    # ln likelihood

def lnprob_Ray(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet, a_IWA):
   
    lp = lnprior_Ray(theta)
    if not np.isfinite(lp):
        return -inf
    
    ll = lnlike(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet,  a_IWA)
    if not np.isfinite(ll):
        return -inf
    
    return lp + ll


In [5]:
# emcee functions for uniform eccentricity dist

def lnprior_uni(theta):
    # The parameters are stored as a vector of values, so unpack them
    lna, cosi, ecc, omega_p, xi_0, lan = theta
    
    # let eccentricity be flat for now
    if not (0 <= ecc < 1) or not (lna_min < lna < lna_max) or not (-1 < cosi < 1) or not (0 < omega_p < 2*np.pi) or not (0 < xi_0 < 2*np.pi) or not (0 < lan < 2*np.pi):
        return -np.inf
    return 0.0
    


def lnprob_uni(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet, a_IWA):
   
    lp = lnprior_uni(theta)
    if not np.isfinite(lp):
        return -inf
    
    ll = lnlike(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet,  a_IWA)
    if not np.isfinite(ll):
        return -inf
    
    return lp + ll


In [6]:
# emcee functions for beta distribution
from scipy.special import gamma
gamma_ab = gamma(beta_a + beta_b)
gamma_a = gamma(beta_a)
gamma_b = gamma(beta_b)
def lnprior_Beta(theta):
    # The parameters are stored as a vector of values, so unpack them
    lna, cosi, ecc, omega_p, xi_0, lan = theta
    
    # let eccentricity be flat for now
    if not (lna_min < lna < lna_max) or not (-1 < cosi < 1) or not (0 < omega_p < 2*pi) or not (0 < xi_0 < 2*pi) or not (0 < lan < 2*pi):
        return -inf
#     if not (0 <= ecc < 1): # bounds of rayleigh distribution
#         return -inf
     

    lnPr_ecc = log(( gamma_ab * ecc**(beta_a-1) * (1-ecc)**(beta_b-1) ) / (gamma_a * gamma_b) )
   
    return lnPr_ecc  # this is normalized (area under pdf = 1)



def lnprob_Beta(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet, a_IWA):
   
    lp = lnprior_Beta(theta)
    if not np.isfinite(lp):
        return -inf
    
    ll = lnlike(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet,  a_IWA)
    if not np.isfinite(ll):
        return -inf
    
    return lp + ll

In [7]:
# from analyze_orbits import read_obvs

# theta = [ 1.17472559,  0.35798731,  0.35423964,  1.87096859,  0.37901581,  3.65733168]
# x, y, aproj = read_obvs('data/90day_nov28')[8]
# x = x[:2]
# y=y[:2]
# aproj=aproj[:2]
# cadence=90
# n_epochs=5
# IWA = 3 * 500e-9/10 * 206265 # arcsec
# OWA = 10 * 500e-9/10 * 206265 # arcsec
# d=10
# s = d*(5*1e-3) # convert mas error to AU
# a_IWA = d*IWA
# i_nondet = []
# i_det = [0, 1]

# lnprob = lnprob_Beta(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet, a_IWA)
# ll = lnlike(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet,  a_IWA)
# print('ln prob', lnprob)
# print('ln likelihood', ll)

In [9]:
# emcee functions for circular orbits - fitting only 4 variables

def lnprior_circ(theta):
    # The parameters are stored as a vector of values, so unpack them
    lna, cosi, xi_0, lan = theta
    
    if not (lna_min < lna < lna_max) or not (-1 < cosi < 1)  or not (0 < xi_0 < 2*np.pi) or not (0 < lan < 2*np.pi):
        return -np.inf
    return 0


def lnprob_circ(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet,  a_IWA):
    lp = lnprior_circ(theta)
    if not np.isfinite(lp):
        return -np.inf
    ll = lnlike(theta, x, y, aproj, s, cadence, n_epochs, i_det, i_nondet,  a_IWA)
    return lp + ll


In [10]:
def read_truths(direc):
    cwd = os.getcwd()
    f = open(str(cwd)+'/'+direc+'/truths_iters.dat', 'rb')
    s = pickle.load(f)  # variables come out in the order you put them in
    f.close()
    return s

def read_obvs(direc):
    cwd = os.getcwd()
    f = open(str(cwd)+'/'+direc+'/obvs_iters.dat', 'rb')
    s = pickle.load(f)  # variables come out in the order you put them in
    f.close()
    return s

In [11]:
def obv_at_epoch(truths, n_epoch, cadence=None, s_AU=0.01, noise=False): 
    
    try:
        lna, cosi, ecc, omega_p, xi_0, lan = truths
    except: # circular orbits, 4 vars
        lna, cosi, xi_0, lan = truths
        ecc=0
        omega_p=0
        
    
    a = exp(lna)
    b = a*sqrt(1-ecc**2)
    c = sqrt(a**2 - b**2) # ellipse

    if n_epoch==0:
        xi = xi_0
    else:
#         calculate xi: assume that planet has been moving on Keplerian orbit given delta t, sma posterior
#         the phase the planet would be observed at this epoch given these parameters
        xi = getNextPhase(n_epoch, cadence, a, xi_0)

    alpha = lan #lan
    beta = np.arccos(cosi)
    gamma = omega_p
    t = xi
    x, y, _ = EuclideanRotation(alpha, beta, gamma, a, b, t) # this takes iterables for t
    
    if noise:
        x, y = np.random.normal(loc=[x, y], scale=s_AU/sqrt(2))   

    # calculate euclidean distance 
    aproj = sqrt(x**2 + (y**2))

    return x, y, aproj

from math import sin, cos
def EuclideanRotation(alpha, beta, gamma, a, b, t):
        # t can be an iterable
    c = sqrt(a**2 - b**2) # ellipse
    try:
        v = np.array([[a*cos(t) - c],[b*sin(t)],[0]]) # column vector of points
    except TypeError: 
        v = np.array([[a*np.cos(t) - c],[b*np.sin(t)],[0]]) # column vector of points
    cA = cos(alpha)
    cB = cos(beta)
    cC = cos(gamma)
    
    sA = sin(alpha)
    sB = sin(beta)
    sC = sin(gamma)
    
    R = np.array([[cA*cC - cB*sA*sC, -cC*sA-cA*cB*sC, sB*sC],
                 [cB*cC*sA + cA*sC, cA*cB*cC - sA*sC, -cC*sB],
                 [sA*sB, cA*sB, cB]])
    
#     R = np.array([[np.cos(beta), -np.cos(gamma)*np.sin(beta), np.sin(beta)*np.sin(gamma)], 
#                   [np.cos(alpha)*np.sin(beta), np.cos(alpha)*np.cos(beta)*np.cos(gamma)-(np.sin(alpha)*np.sin(gamma)), -np.cos(gamma)*np.sin(alpha)-(np.cos(alpha)*np.cos(beta)*np.sin(gamma))],
#                   [np.sin(alpha)*np.sin(beta), np.cos(alpha)*np.sin(gamma)+(np.cos(beta)*np.cos(gamma)*np.sin(alpha)), np.cos(alpha)*np.cos(gamma)-(np.cos(beta)*np.sin(alpha)*np.sin(gamma))]])
    
    L_rot = np.dot(R, v)
    x = L_rot[0]
    y = L_rot[1]
    z = L_rot[2]
    return x[0], y[0], z[0]

In [64]:
# truths = [np.log(1.0), np.cos(1.08312938748), 0.378364330327, 1.70787590579, 2.13710086603, 1.66472380962]
# aproj_= []
# _, _, aproj_true = obv_at_epoch(truths, 0, 90, noise=False)
# for ii in range(1000):
#     _, _, aproj = obv_at_epoch(truths, 0, 90, s_AU=0.05, noise=True)
#     aproj_.append(aproj)
    
# print('true', aproj_true)
# print(np.mean(aproj_))
# print(np.std(aproj_))
# print('range', np.max(aproj_) - np.min(aproj_))
# print(np.max(aproj_))
# print(np.min(aproj_))

true 0.831481967234
0.831704410816
0.0356001032434
range 0.219919731975
0.955725525915
0.73580579394


In [8]:
# alt functions for emcee where all orbits are circular

def lnprior_circular(theta):
    # The parameters are stored as a vector of values, so unpack them
    lna, cosi, omega_p, xi_0 = theta
    
    # let eccentricity be flat for now
    if not (lna_min < lna < lna_max) or not (0 < cosi < 1) or not (0 < omega_p < 2*np.pi) or not (0 < xi_0 < 2*np.pi):
        return -np.inf

    return 0 

def lnlike_circular(theta, aproj, s, cadence):
    # fitting aproj(xi) with several measurements of aproj given as a list
    
    #lna, cosi, ecc, omega_p, xi = theta
    #model = aproj(ecc, np.exp(lna), np.arccos(cosi), xi, omega_p) # returns a_proj
    lna, cosi, omega_p, xi_0 = theta
    n_epochs = len(aproj)
    el = np.zeros(n_epochs) # chi sqr terms to be summed
    # get error for each aproj measurement
#     for n in range(n_epochs):
#         expected = aproj_at_epoch(ecc, np.exp(lna), np.arccos(cosi), omega_p, xi_0, n, cadence) # returns a_proj
#         el[n] = ( aproj[n] - expected )**2 / (2*s**2)
    expected = aproj_at_epoch(0, np.exp(lna), np.arccos(cosi), omega_p, xi_0, range(n_epochs), cadence) # returns a_proj
    el = ( np.array(aproj) - np.array(expected) )**2 / (2*s**2)
    return -np.sum(el)
    
    #return -0.5*(np.sum( ((aproj-model)/s)**2. ))

def lnprob_circular(theta, aproj, s, n_epoch, cadence):
    lp = lnprior_circular(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike_circular(theta, aproj, s, cadence)


In [1]:
# functions for running
import json
import scipy.optimize as op
from analyze_orbits import maxlikelihood, plot_chain, read_results


# @profile(print_stats=10, sort_stats='time')
def run_epochs(n_epochs_tot, truths, obvs, s, cadence, nwalkers=100, saveall=False, ecc_dist='Rayleigh',
               fname='samples', progress=True, chainlength=1e5, a_IWA=None, ep_start=0, d=None, it_start=0,
               i_it=None, direc=None, show_autocorr=False, show_chainplot=False, 
               initialize='ML', analytic=False):
   
    # loops over each epoch to get their posteriors, each set of samples represents the cumulative information over the preceeding epochs
    # cadence: days, s: measurement error, pos, nwalkers from emcee
    cwd = os.getcwd()
    ndim = len(truths)
    samples_epochs = [] # mcmc output will be stored here
    lnprob_epochs = []
    chains_epochs = []
    flatchains_epochs=[]
    ML_epochs = []
    std_ep = []
    mean_ep =[]

    if ecc_dist=='circular':
        lna_true, cosi_true, xi_0_true, lan_true = truths
    #         ecc_true=0
    #         omega_p_true=0
        pos_min = np.array([lna_min, 0., 0., 0.])
        pos_max = np.array([lna_max, 1., 2*np.pi, 2*np.pi])        

    else:
        lna_true, cosi_true, ecc_true, omega_p_true, xi_0_true, lan_true = truths
#         pos_min = np.array([lna_min, 0., 0., 0., 0., 0.])
#         pos_max = np.array([lna_max, 1., 1., np.pi, np.pi, np.pi])
        pos_min = np.array([lna_min, -1., 0., 0., 0., 0.])
        pos_max = np.array([lna_max, 1., 1., 2*np.pi, 2*np.pi, 2*np.pi])

#     psize = pos_max - pos_min
#     pmedian = np.median([pos_min, pos_max], axis=0)
    pmedian = [-0.53647227,  0.5,   0.0001,  1.57079633,  1.57079633,  1.57079633]
    psize = pos_max - pos_min
    
    x_true=[]
    y_true=[]
    aproj_true = [] # list of a_proj observations
    i_det=[]
    i_nondet=[]
    
    x_eps, y_eps, aproj_eps = obvs # unpack - these have added noise
    
    for n in range(n_epochs_tot):
        print('\n\n\n     epoch',n,'/',n_epochs_tot)
        x_clean, y_clean, aproj_clean =  obv_at_epoch(truths, n_epoch=n, cadence=cadence, s_AU=s, noise=False)  
        x = x_eps[n]
        y = y_eps[n]
        aproj = aproj_eps[n]
        x_true.append(x)
        y_true.append(y)
        aproj_true.append(aproj)
        print('          (x, y, aproj) noisy =', x, y, aproj)
        print('          (x, y, aproj) clean =', x_clean, y_clean, aproj_clean)


        if aproj > a_IWA:
            i_det.append(n)
            det_text = 'detection'
        else:
            i_nondet.append(n)
            det_text = 'nondetection'
        print('          '+det_text)

   
        if n >= ep_start: # so you can just fit the three epochs at the end etc
            print('\n          truths', truths[0:3])
            if n >= 3:
                k = nwalkers
                steps = chainlength
            elif (n < 2) and analytic:
                k=13
                steps = 1050
            elif (n==0) and (ep_start==0):
                # first epoch
                k = nwalkers*2
                steps = chainlength*10
            else:
                # double walkers for epoch 3
                k = nwalkers*2
                steps = chainlength
            
            if ecc_dist=='Beta':
                nll = lambda *args: -lnprob_Beta(*args)
            elif ecc_dist=='uniform':
                nll = lambda *args: -lnprob_uni(*args)
            if initialize=='ML':
                # get starting guess --> https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
                print('          initialize ML')
                result = op.minimize(nll, [truths], args=(x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA))
                theta_ml = result["x"]
    #             print('          theta_ml % error:' (theta_ml[0:3]-truths[0:3])/theta_ml[0:3])
                pos = [result["x"] + 1e-1*np.random.randn(ndim) for i in range(k)]
            elif initialize=='aproj':
                print('          initalize aproj')
                inittruth = pmedian
                inittruth[0] = aproj
                result = op.minimize(nll, [inittruth], args=(x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA))
                theta_ml = result["x"]
                print('          theta_ml', theta_ml[0:3])
                pos = [result["x"] + 1e-2*np.random.randn(ndim) for i in range(k)]
            elif initialize=='random':
                print('          initialize random')
                pos = [pos_min + psize*np.random.rand(ndim) for i in range(k)]
            elif initialize=='last':
                # start with random guess for epoch 0
                if n==ep_start:
                    print('          initialize random')
                    pos = [pos_min + psize*np.random.rand(ndim) for i in range(k)]
                else:
                    print('          initialize from last epoch -->', n-1)
                    # retrieve last epoch's ML
                    init = ML_new.copy()
                    print('ML_new', ML_new)
                    init[0] = np.log(init[0])
                    init[1] = np.cos(init[1])
                    print('init', init)
                    print('ML_new', ML_new)
                    pos = [init + 1e-1*np.random.randn(ndim) for i in range(k)]
                    
            else:
                print('need initialization')
            
                
            # correct for out of bounds
            # PROBLEM: if spread of ball is too wide, and walkers are forced out of range, 
            # then u get lots of things at 0 or 1
            for ik in range(k):
#                 if pos[ik][0] < 0: # ln(a)
#                     pos[ik][0] = 0
                if pos[ik][1] < 0: # cos(i)
                    pos[ik][1] = 0
                elif pos[ik][1] > 1:
                    pos[ik][1] = 1
                if pos[ik][2] < 0: # e
                    pos[ik][2] = 0
                elif pos[ik][2] > 1:
                    pos[ik][2] = 1
       
    
#             pos = [truths + 1e-1*np.random.randn(ndim) for i in range(k)]
#             print('pos', np.shape(pos))

            # run shit
         
            if ecc_dist=='Rayleigh':
                sampler = emcee.EnsembleSampler(k, ndim, lnprob_Ray, 
                                                args=[x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA]) 
            elif ecc_dist=='Beta':
                sampler = emcee.EnsembleSampler(k, ndim, lnprob_Beta, 
                                                args=[x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA]) 
            elif ecc_dist=='uniform':
                sampler = emcee.EnsembleSampler(k, ndim, lnprob_uni, 
                                                args=[x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA]) 
            elif ecc_dist=='circular':
                sampler = emcee.EnsembleSampler(k, ndim, lnprob_circ, 
                                                args=[x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA]) 


            sampler.run_mcmc(pos, steps, progress=progress) # chain shape (n_walkers, n_steps-1000, ndim)
            
#             print('lnprobability shape', np.shape(sampler.lnprobability))
#             print('lnprobability[1000:] shape', np.shape(sampler.lnprobability[1000:]))
            
            # store shit
            samples = sampler.chain[:, 1000:, :].reshape((-1, ndim)) # discard first 1000 steps
            lnprob = sampler.lnprobability
#             print('samples id', id(samples[0,0]))
           
            chains = sampler.chain
#             print('steps', steps)
#             print('chains', np.shape(chains))
            chains_epochs.append(chains)
#             print('chains id', id(chains[0,0,0]))
            
            
            
            std_ep.append(np.std(samples, axis=0))
            mean_ep.append(np.mean(samples, axis=0))

            samples_epochs.append(samples)
            lnprob_epochs.append(lnprob)
            
            
            # save max likelihood fit
            flatchain = sampler.flatchain
            flatprob = sampler.flatlnprobability
            
            maxes = np.argwhere(flatprob == np.amax(flatprob))
            print('\n          # MLs', len(maxes))
        
#             print('flatchain id', id(flatchain[0,0]))
#             print('flatchain[1]', flatchain[1])
            
#             flatchain0 = chains.reshape((-1, ndim),  order='F')
#             print('flatchain0 id',id(flatchain0[0,0]))
#             flatprob0 = lnprob.flatten()
#             print('flatchain=flattened chain', (flatchain==flatchain0).all())
#             print('flatprob=flattened prob', (flatprob==flatprob0).all())
            

            ML_new, lnprob, i_ML = maxlikelihood(flatchain, flatprob, 
                                                  x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA,
                                                 returnmore=True, multiple=True,
                                               )
            print('\n          ML_retrieved',lnprob,'@',i_ML)
     
            print('--------->theta_ML:',ML_new)
#             print('          lnprob inputs:', ML_new, x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA)
            print('\n          lnprob_forward:',lnprob_Beta(ML_new, x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA))
            
        
        
        
#             ML_new0, lnprob0, i_ML0 = maxlikelihood(flatchain0, flatprob0, returnmore=True)
#             print('\n          ML0_retrieved',lnprob0,'@',i_ML0)
     
#             print('          theta_ML0:',ML_new0)
        
#             print('\n        lnprob0_forward:',lnprob_Beta(ML_new0, x_true, y_true, aproj_true, s, cadence, n+1, i_det, i_nondet, a_IWA))
            
#             print('          i_ML',np.where(flatprob == lnprob))
        
            # need to reparameterize to match earlier 
            ML_new[0] = np.exp(ML_new[0])
            ML_new[1] = np.arccos(ML_new[1])
            ML_epochs.append(ML_new)
            
            flatchains_epochs.append(flatchain)
                

            print("          Mean acceptance fraction: {0:.3f}"
                    .format(np.mean(sampler.acceptance_fraction)))
             
            if show_autocorr:
                
                try:
                    print("          Autocorrelation time: ", sampler.get_autocorr_time())
        #                 plot_convergence(sampler, n)
                except Exception as e:
                    print(e)

        else: 
            
            # load previous results
            
           
            try:
                f = open(fname+'.dat', 'rb')
                chains = pickle.load(f)  # variables come out in the order you put them in
                f.close()
                chains_epochs.append(chains[n])
                
                f = open(fname+'_lnprob.dat', 'rb')
                lnprob = pickle.load(f)  # variables come out in the order you put them in
                f.close()
                lnprob_epochs.append(lnprob[n])
            except: 
                chains_epochs.append(-99)
                lnprob_epochs.append(-99)
                        
            f = open(str(cwd)+'/'+direc+'/d'+str(d)+'std'+str(i_it)+'.dat', 'rb')
            std = pickle.load(f)  # variables come out in the order you put them in
            f.close()
            std_ep.append(std[n])
            
            f = open(str(cwd)+'/'+direc+'/d'+str(d)+'mean'+str(i_it)+'.dat', 'rb')
            mean = pickle.load(f)  # variables come out in the order you put them in
            f.close()
            mean_ep.append(mean[n])
          
            f = open(str(cwd)+'/'+direc+'/d'+str(d)+'ML'+str(i_it)+'.dat', 'rb')
            ML_new = pickle.load(f)  # variables come out in the order you put them in
            f.close()
            ML_epochs.append(ML_new)
            
            samples_epochs.append(-99*np.ones((int(nwalkers*chainlength), ndim)))
    
        print('n', n)
        print('samples_epochs', np.shape(samples_epochs))
        print('samples_epochs[n]', samples_epochs[n])
    
    #end
    
    if saveall:
        print('\n          pickling to',fname)
        try:
            with open(fname+'.dat', 'wb') as f:
                pickle.dump(chains_epochs, f)
#             with open(fname+'_lnprob.dat', 'wb') as f:
#                 pickle.dump(lnprob_epochs, f)

        except OverflowError:
            print('too much data')
        except MemoryError:
            print('memory error')


    with open(str(cwd)+'/'+direc+'/d'+str(d)+'ML'+str(i_it)+'.dat', 'wb') as f:
        pickle.dump(ML_epochs, f) 
    with open(str(cwd)+'/'+direc+'/d'+str(d)+'std'+str(i_it)+'.dat', 'wb') as f:
        pickle.dump(std_ep, f) 
    with open(str(cwd)+'/'+direc+'/d'+str(d)+'mean'+str(i_it)+'.dat', 'wb') as f:
        pickle.dump(mean_ep, f)
        
    # get fits

    get_orbits(direc, ndim, 1, n_epochs_tot, d, quantiles=[16, 50, 84], 
           i_it=i_it, ep_start=ep_start,
           fname='fitted'+str(i_it)+'.dat', samples_epochs=samples_epochs
              )
    get_orbits(direc, ndim, 1, n_epochs_tot, d, quantiles=[2.5, 50, 97.5],  
           i_it=i_it, ep_start=ep_start,
           fname='fitted'+str(i_it)+'_95.dat', samples_epochs=samples_epochs
              )

#     print('\n N O T    F I T T I N G\n')

    if show_chainplot:
        for n in range(n_epochs_tot):
            fig, ax = plot_chain(direc, i_it, n, 0, d, range(k), 
                       title='Iteration '+str(i_it)+', epoch '+str(n)+': '+det_text, 
                       sigma=1, 
                       lw=0.1, alpha=0.9)
            fig.show()

    return samples_epochs


def run_iterations(n_iter, n_epochs_tot, s_mas, cadence, nwalkers=100, iter_start=0, direc='', 
                   saveall=False, ecc_dist='Rayleigh', ecc_dist_true='Rayleigh', 
                   D_tel=10, wl=500e-9,a_fixed=None, stem='samples_epochs', 
                   progress=True, chainlength=1e5, show_chainplot=False,analytic=False,
                   truths_iters=None, newtruths=False, newobvs=False, overwrite=False, forcenondetect=None,
                   d=10, ep_start=0, show_autocorr=False, force_first_detection=True, initialize='ML',
                  n_iter_total=50):
    
    
    # truths_iters is a list containing values for each iteration
    # n_iter is a scalar
    print('lna_min', lna_min)
    print('lna_max', lna_max)
    cwd = os.getcwd()
    
    IWA = 3 * wl/D_tel * 206265 # arcsec
    OWA = 10 * wl/D_tel * 206265 # arcsec
    print('distance at which a_IWA=1.7 AU :', 1.7/IWA, 'pc')

    s_AU = d*(s_mas*1e-3) # convert mas error to AU
    a_IWA = d*IWA

    print("s_xy =", s_AU)
    print('a_IWA =',  a_IWA)
    
#     x0 = 2.5 *wl/D_tel * 206265 * d # AU
#     b =1
    
#     if d==20:
#         c = 0.000000001
#         norm = 0.515663
#     elif d==10:
#         c = 0.0000000000000001
#         norm = 0.257833
#     elif d==5:
#         c = 0.00000000000000000000000000000001
#         norm = 0.128917

#     f_lnlike_nondet = lambda x: np.log( 1 / (1 + b*c**(x0-np.array(x)))  /  norm )
    
    
    if len(direc) > 0:
        if not os.path.exists(direc):
            os.makedirs(direc)
            
    if (truths_iters is None):
        truths_file = Path(direc+'/truths_iters.dat')
        if not newtruths:
            if truths_file.exists():
                truths_iters = read_truths(direc)
                print('\nreading truths file from ',direc)
                if not newobvs:
                    obvs_iters = read_obvs(direc)
                    print('\nreading obvs file from ',direc)
                
            else:
                print('\nnew truths?')
                return(None)
        else:
            if truths_file.exists() and (not overwrite):
                print('\nWARNING: truths file exists, to overwrite set overwrite=True')
                return(None)
            else:
                print('\ngenerating new truths for',n_iter,'iteration(s)')
                truths_iters, obvs_iters = get_random_orbit(n_iter_total,
                                                            cadence=cadence, s_xy = s_AU, 
                                                 ecc_dist_true=ecc_dist_true, a_IWA=a_IWA,
                                                a_fixed=a_fixed, i_miss=forcenondetect, 
                                                            force_first_detection=force_first_detection,
                                                           n_ep=n_epochs_tot)
    
    else:
        if newtruths:
            print('\ntruths_iters must be None to get new truths. continuing with input truths')
        if np.shape(truths_iters)[0] < n_iter:
            print('\ntoo many iterations for input truth list')

    if newtruths:
        # store truths
        
        with open(str(cwd)+'/'+direc+'/truths_iters.dat', 'wb') as f:
            pickle.dump(truths_iters, f)  
        with open(str(cwd)+'/'+direc+'/obvs_iters.dat', 'wb') as f:
            pickle.dump(obvs_iters, f)  
            
    if newobvs:
        obvs_iters=[]
        if newtruths:
            print('generating new obvs from new truths')
        else:
            print('generating new obvs from input truths for entire range')
        for ii in range(n_iter_total):
            obvs_iters.append(obv_at_epoch(truths_iters[ii], n_epoch=range(n_epochs_tot), cadence=cadence, 
                                       s_AU=s_AU, noise=True))
        # store
        with open(str(cwd)+'/'+direc+'/obvs_iters.dat', 'wb') as f:
            pickle.dump(obvs_iters, f) 
    
    ndim = len(truths_iters[0])
    for ii in range(iter_start, n_iter):
        print('\n ///////////// ( • ᴗ•)  /////////////////////////////////')
        print('\n\niteration ',ii,'/',n_iter)
        fname='/'+direc+'/'+stem+str(ii)
        cwd = os.getcwd()
        samples_epochs = run_epochs(n_epochs_tot, truths_iters[ii], obvs_iters[ii], s_AU, 
                                    cadence,saveall=saveall,nwalkers=nwalkers, 
                                    fname=str(cwd)+fname, progress=progress, analytic=analytic,
                                    chainlength=chainlength, a_IWA=a_IWA, 
                                    ep_start=ep_start, ecc_dist=ecc_dist, it_start=iter_start,
                                    d=d, i_it=ii, show_chainplot=show_chainplot,
                                    direc=direc, show_autocorr=show_autocorr, initialize=initialize)

#         if not saveall:
        
    
#         get_orbits(direc, ndim, 1, n_epochs_tot, d, quantiles=[16, 50, 84], i_it=ii,
#                    fname='fitted'+str(ii)+'.dat', samples_epochs=samples_epochs)
#         get_orbits(direc, ndim, 1, n_epochs_tot, d, quantiles=[2.5, 50, 97.5],  i_it=ii,
#                    fname='fitted'+str(ii)+'_95.dat', samples_epochs=samples_epochs)


    overwrite=False
    return overwrite

ModuleNotFoundError: No module named 'analyze_orbits'

In [15]:
def get_random_orbit(n, s_xy, a_fixed=None,  ecc_dist_true='Rayleigh', i_miss=None, 
                     force_first_detection=True,
                     cadence=None, a_IWA=None, n_ep=None):
    # ecc_dist: 'linear', 'circular', 'Rayleigh'
    # forcenondetect = (0, 1) <-- forces first two epochs to be nondetections
    truths_iters=[]
    obvs_iters = []

    for ii in range(n): # how many planets to simulate
        truths=randorbit(a_fixed, ecc_dist_true)

        flag=False
        flag_onlyfirst=False
        flag_onlymiss=False
        
        
        if (i_miss is not None) and force_first_detection:
            # forcing nondetection 
            flag=True   
            if ii==0:
                print('forcing nondetection @', i_miss)
                print('forcing detection at epoch 0')
        elif i_miss is not None:
            flag_onlymiss=True
            if ii==0:
                print('forcing nondetection @', i_miss)
        elif force_first_detection:
            flag_onlyfirst=True
            if ii==0:
                print('forcing detection at epoch 0')
        else:
            x, y, aproj = obv_at_epoch(truths, n_epoch=range(n_ep), cadence=cadence, 
                                       s_AU=s_xy, noise=True)
            
        while flag:
            x, y, aproj = obv_at_epoch(truths, n_epoch=range(n_ep), cadence=cadence, 
                                       s_AU=s_xy, noise=True)

            if (aproj[i_miss]<=a_IWA) and (aproj[0]>a_IWA): # nondetection @ forced and first epoch is detection
                break
            truths=randorbit(a_fixed, ecc_dist_true)
         
        while flag_onlymiss:
            x, y, aproj = obv_at_epoch(truths, n_epoch=range(n_ep), cadence=cadence, 
                                       s_AU=s_xy, noise=True)
            if aproj[i_miss]<=a_IWA:
                break
            truths=randorbit(a_fixed, ecc_dist_true)
        
        while flag_onlyfirst:    
            x, y, aproj = obv_at_epoch(truths, n_epoch=range(n_ep), cadence=cadence, 
                                       s_AU=s_xy, noise=True)
            if aproj[0]>a_IWA: # nondetection @ forced and first epoch is detection
                break
            truths=randorbit(a_fixed,  ecc_dist_true)
            
        truths_iters.append(truths) 
        obvs_iters.append((x, y, aproj))
 
    return truths_iters, obvs_iters

def randorbit(a_fixed=None, ecc_dist_true='Rayleigh'):
    lna_true = getlnSemimajorAxis(lna_min_true, lna_max_true, a_fixed=a_fixed)
    cosi_true = np.cos(getInclination())
    omega_p_true, xi_0_true, lan_true = random_uniform_circle(n=3)
    if ecc_dist_true=='circular':
        return (lna_true, cosi_true, xi_0_true, lan_true)
    else:
        ecc_true = getEccentricity(ecc_dist_true) 
#     print('lna_true', lna_true)
    return (lna_true, cosi_true, ecc_true, omega_p_true, xi_0_true, lan_true)

In [11]:
def get_random_orbit_given_obvs(n, s_xy, a_fixed=None, ecc_dist='Rayleigh', ecc_dist_true='Rayleigh', 
                                tol=1e-2, aproj_fixed=1,i_ep=None,
                                 cadence=None, a_IWA=None, n_ep=None):
    # ecc_dist: 'linear', 'circular', 'Rayleigh'
    # forcenondetect = (0, 1) <-- forces first two epochs to be nondetections
    truths_iters=[]
    obvs_iters = []

    for ii in range(n): # how many planets to simulate
        truths = randorbit(a_fixed, ecc_dist, ecc_dist_true)
        x, y, aproj = obv_at_epoch(truths, n_epoch=range(n_ep), cadence=cadence, 
                                    s_AU=s_xy, noise=True)
        flag=False

        
        if abs(aproj[i_ep]-aproj_fixed) > tol:
            # forcing nondetection 
            flag=True   
            if ii==0:
                print('forcing a_proj @', aproj_fixed)

            
        while flag:
            x, y, aproj = obv_at_epoch(truths, n_epoch=range(n_ep), cadence=cadence, 
                                       s_AU=s_xy, noise=True)

            if abs(aproj[i_ep]-aproj_fixed) < tol: # nondetection @ forced and first epoch is detection
                break
                
            truths=randorbit(a_fixed, ecc_dist, ecc_dist_true)
            truths_iters.append(truths) 
            
        obvs_iters.append((x, y, aproj))
 
    return truths_iters, obvs_iters


In [13]:
def getNextPhase(n, cadence, smas, phases):
    # this works with iterables or scalars
    # cadence is fixed in units day
    
    # calculate planet speed
    meanmotion = np.sqrt(mu_earth/(smas*AU2m)**3) # units rad/s
    deltat = meanmotion*(np.array(n)*(cadence*86400)) # delta M
    M_f = np.add(phases, deltat)

    try:
        newphase = float(M_f) % (2*np.pi)
    except TypeError: 
        # M_f is iterable
        newphase = [float(s) % (2*np.pi) for s in M_f] 
        
        
#     if np.size(M_f)!=1:
#         newphase = [float(s) % (2*np.pi) for s in M_f] 
#     else:
#         newphase = float(M_f) % (2*np.pi)
    return newphase

In [14]:
# generate planetary parameters
  
def getlnSemimajorAxis(lnmin, lnmax, a_fixed=None):
    if a_fixed is None:
        return np.random.uniform(low=lnmin, high=lnmax, size=None) 
    else:
        return np.log(a_fixed)
    
def getInclination():
    cosi = np.random.uniform(low=0, high=1, size=None) # uniform in cos(i)
    return np.arccos(cosi)

def getEccentricity(dist):
    #return np.random.uniform(0, 0.5) # worst case
    if dist=='Rayleigh':
        sigma = 0.081
        sigma_low = 0.01 # 90% of planets come from this
        sigma_high=0.22 # 10% of planets come from this
#         return rayleigh.rvs(size=None, scale=sigma)
        return np.random.rayleigh(scale=sigma) # Shabram+ 2015
    elif dist=='Beta':
        from scipy.stats import beta
        # beta dist from Hogg et al. (2010) and Kipping (2013)
        a = 0.867 
        b = 3.03
        r = beta.rvs(a, b, size=1)
        return r[0]
    elif dist=='linear':
        # slope of -2.18
        # actually try slope of 1
        return np.random.triangular(left=0, right=1, mode=0)
    elif dist=='uniform':
        return np.random.uniform(low=0, high=1)
    elif dist=='circular':
        return 0
    
def random_uniform_circle(n=1): 
    # uniform in linear
    return np.random.uniform(low=0, high=np.pi, size=n)


In [14]:
def auto_window(taus, c):
    m = np.arange(len(taus)) < c * taus
    if np.any(m):
        return np.argmin(m)
    return len(taus) - 1

def autocorr_new(y, c=5.0):
    f = np.zeros(y.shape[1])
    for yy in y:
        f += emcee.autocorr_func_1d(yy)
    f /= len(y)
    taus = 2.0*np.cumsum(f)-1.0
    window = auto_window(taus, c)
    return taus[window]